In [1]:
import common
import importlib
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import socket

# Show all columns and rows in a dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [18]:
# graphalytics inserts
pers_experiments = "num_threads_read == 0 and (hostname == 'scyper21' or hostname == 'scyper22')"

data_ins = common.import_gfe("view_graphalytics_inserts").query(pers_experiments).copy() # data from the experiments
data_ins["build_frequency"].fillna(pd.Timedelta(0), inplace=True) # replace NaT with 0, otherwise the records are ignored in the group by

data_ins = data_ins.query("((block_size == 512) and library == 'sortledton') or library != 'sortledton'")

# graphalytics load
data_load = common.import_gfe("view_graphalytics_load").query(pers_experiments).copy()
# shape the dataframe as data_ins
data_load["build_frequency"] = pd.Timedelta(0)
data_load["build_frequency_secs"] = np.NaN
data_load["num_snapshots_created"] = 0
data_load["num_threads_write"] = 0
data_load["block_size"] = -1

data = pd.concat([data_ins, data_load])
data = data.groupby(["library", "compiler_family", "graph", "build_frequency","num_threads_read", "num_threads_write", "algorithm"]) \
    .agg(completion_time=("median_secs", "median"), count=("median_secs", "count"))

data = data.unstack("algorithm")[("completion_time")]
data.index.set_names("compiler", level=1, inplace=True)

List of available executions:

In [19]:
fn_join_threads = lambda grp: ", ".join(map(str, grp.unique()))
data.reset_index().groupby(["library", "compiler"]).agg(num_threads_w=("num_threads_write", fn_join_threads), num_threads_r=("num_threads_read", fn_join_threads))

,,num_threads_w,num_threads_r
library,compiler,,
csr3,gcc,0,0
csr3-lcc,gcc,0,0
g1_v6-ref-ignore-build,gcc,20,0
livegraph3_ro,gcc,20,0
llama8-ref,gcc,16,0
sortledton,gcc,56,0
stinger7-ref,gcc,56,0
teseo-lcc-dv.12b,gcc,56,0
teseo-lcc.12,gcc,56,0


### CSR

In [20]:
csr = data.loc[("csr3-lcc", "gcc")].droplevel([1,2,3]).rename(columns = {"lcc": "lcc_sorted"})
csr["lcc_std"] = data.loc[("csr3", "gcc")].droplevel([1,2,3]).rename(columns = {"lcc": "lcc_std"})["lcc_std"]
csr = csr[["bfs", "cdlp", "lcc_std", "lcc_sorted", "pagerank", "sssp", "wcc"]] # reoder the columns
csr = pd.concat([csr], names=["library"], keys=["csr"]) # prepend the key `csr`

In [5]:
csr

algorithm                    bfs        cdlp      lcc_std  lcc_sorted  \
library graph                                                           
csr     com-friendster  1.007024  179.372491   835.651810   67.085626   
        dota-league     0.002061    1.116481   363.353080   53.038842   
        graph500-22     0.034984    7.841334   306.267269   12.374704   
        graph500-24     0.133839   24.633421  2236.280409   76.673949   
        graph500-26     0.490420  142.827060          NaN  587.403737   
        uniform-24      0.147175   31.715860    21.949081    2.933300   
        uniform-26      0.581879  176.101286    92.364151   13.929533   

algorithm                pagerank       sssp        wcc  
library graph                                            
csr     com-friendster  31.575244  53.162582  10.208290  
        dota-league      0.157165   0.434347   0.032011  
        graph500-22      0.409857   0.915066   0.158914  
        graph500-24      2.718796   5.512071   0.925525  
        graph500-26     15.290341  36.776917   4.950484  
        uniform-24       3.445015   7.026035   1.174713  
        uniform-26      18.702736  36.400172   6.212963

### Other systems

In [21]:
stinger = pd.concat( [
    data.loc[("stinger7-ref", "gcc")].droplevel([1, 2, 3])
], names=["library"], keys=["stinger"])
llama = pd.concat( [
    data.loc[("llama8-ref", "gcc", slice(None), "00:00:10", 0, 16)].droplevel([0, 1, 3, 4, 5]) # dynamic scheduling in OpenMP
], names=["library"], keys=["llama"] )
graphone = pd.concat( [
   data.loc[("g1_v6-ref-ignore-build", "gcc", slice(None), slice(None), 0, 20)].droplevel([0, 1, 3, 4, 5])
], names=["library"], keys=["graphone"] )
livegraph = pd.concat( [
    data.loc[("livegraph3_ro", "gcc", slice(None), slice(None), slice(None), 20)].droplevel([0, 1, 3, 4, 5])
], names=["library"], keys=["livegraph"] )
competitors = pd.concat([stinger,llama,graphone, livegraph]).rename(columns = {"lcc": "lcc_std"})
#competitors = pd.concat([stinger]).rename(columns = {"lcc": "lcc_std"})
competitors["lcc_sorted"] = np.NaN

### Teseo

In [7]:
teseo_lcc = data.loc[("teseo-lcc.12", "gcc")].droplevel([1,2,3]).rename(columns={"lcc": "lcc_sorted"})
teseo_dv_lcc = data.loc[("teseo-lcc-dv.12b", "gcc")].droplevel([1,2,3]).rename(columns={"lcc": "lcc_sorted"})
teseo_lcc["lcc_std"] = np.NaN
teseo_dv_lcc["lcc_std"] = np.NaN

#teseo_std = data.loc[("teseo.12", "gcc")].droplevel([1,2,3]).rename(columns={"lcc": "lcc_std"})
#teseo_std_dv = data.loc[("teseo-dv.12b", "gcc")].droplevel([1,2,3]).rename(columns={"lcc": "lcc_std"})
teseo_logical = pd.concat([ pd.concat([teseo_lcc], axis=1 ) ], names=["library"], keys=["teseo_logical"])
teseo_real = pd.concat([ pd.concat([teseo_dv_lcc], axis=1 ) ], names=["library"], keys=["teseo_real"])

### Sortledton

In [22]:
sortledton1 = data.loc[("sortledton", "gcc")].droplevel([1,2,3]).rename(columns={"lcc": "lcc_sorted"})
sortledton = pd.concat([ pd.concat([sortledton1], axis=1 ) ], names=["library"], keys=["sortledton"])
sortledton

algorithm                       bfs        cdlp   lcc_sorted   pagerank  \
library    graph                                                          
sortledton com-friendster  3.200434  250.072451   137.252343  34.671289   
           dota-league     0.003543    1.204768   429.506597   0.125936   
           graph500-22     0.106116    6.123702    45.544339   0.449037   
           graph500-24     0.418326   24.161871   229.016381   2.299000   
           graph500-26     1.527922  148.985451  1258.009455  11.994295   
           uniform-24      0.409897   35.095181     5.073461   4.033271   
           uniform-26      1.517634  157.081932    22.515912  20.744740   

algorithm                       sssp        wcc  
library    graph                                 
sortledton com-friendster  65.692705  16.556693  
           dota-league      0.409657   0.027747  
           graph500-22      1.165284   0.132754  
           graph500-24      5.537105   0.643460  
           graph500-26     33.926792   3.105470  
           uniform-24      11.302015   1.061742  
           uniform-26      56.170403   7.690647

### Dataset

In [23]:
ds = pd.concat([csr, competitors, teseo_logical, teseo_real, sortledton])
ds = pd.concat([ds, pd.concat([ds.query("library != 'csr'").groupby(by="graph", axis=0).apply(lambda t: t.min())], names=["library"], keys=["best_excl_csr"])])
ds

algorithm                          bfs        cdlp      lcc_std   lcc_sorted  \
library       graph                                                            
csr           com-friendster  1.007024  179.372491   835.651810    67.085626   
              dota-league     0.002061    1.116481   363.353080    53.038842   
              graph500-22     0.034984    7.841334   306.267269    12.374704   
              graph500-24     0.133839   24.633421  2236.280409    76.673949   
              graph500-26     0.490420  142.827060          NaN   587.403737   
              uniform-24      0.147175   31.715860    21.949081     2.933300   
              uniform-26      0.581879  176.101286    92.364151    13.929533   
stinger       dota-league     0.123221    1.557826  1031.744225          NaN   
              graph500-22     0.141997    4.144559   898.389922          NaN   
              graph500-24     0.510672   18.533248          NaN          NaN   
              uniform-24      0.283319   25.222787    53.306832          NaN   
llama         dota-league     0.003092    2.576757          NaN          NaN   
              graph500-22     0.044140    7.694435          NaN          NaN   
              graph500-24     0.161194   30.415791          NaN          NaN   
              uniform-24      0.209384   63.223419          NaN          NaN   
graphone      dota-league     3.315888    4.678014   561.143204          NaN   
              graph500-22     3.323561   10.008789   461.433619          NaN   
              graph500-24     3.782380   26.388428          NaN          NaN   
              graph500-26     4.113198  123.767266          NaN          NaN   
              uniform-24      3.523884   39.322918    71.744946          NaN   
              uniform-26      5.315997  188.698372   338.473133          NaN   
livegraph     com-friendster  8.385851  192.328403  1390.234754          NaN   
              dota-league     0.060769    1.387811   566.778569          NaN   
              graph500-22     0.529737    7.347029   476.949688          NaN   
              graph500-24     2.185411   22.184035  3533.842373          NaN   
              graph500-26     9.019909  116.686941          NaN          NaN   
              uniform-24      0.800779   44.756946    37.466881          NaN   
              uniform-26      3.152725  108.645120   162.956549          NaN   
teseo_logical com-friendster  1.406545  413.898738          NaN   407.803028   
              dota-league     0.010583    1.680501          NaN    88.009714   
              graph500-22     0.053457    4.796505          NaN    40.657832   
              graph500-24     0.242639   27.420749          NaN   418.848337   
              graph500-26     0.580025  136.580121          NaN          NaN   
              uniform-24      0.159577   42.991048          NaN    13.535823   
              uniform-26      0.573264  192.587588          NaN    70.428172   
teseo_real    dota-league     0.010909    1.156862          NaN    77.715276   
              graph500-22     0.045070    3.551794          NaN    40.914416   
              graph500-24     0.191441   14.287923          NaN   153.245635   
              graph500-26     0.416275   62.333133          NaN   928.918946   
              uniform-24      0.110974   24.871898          NaN     6.395665   
              uniform-26      0.414994  113.685187          NaN    28.647182   
sortledton    com-friendster  3.200434  250.072451          NaN   137.252343   
              dota-league     0.003543    1.204768          NaN   429.506597   
              graph500-22     0.106116    6.123702          NaN    45.544339   
              graph500-24     0.418326   24.161871          NaN   229.016381   
              graph500-26     1.527922  148.985451          NaN  1258.009455   
              uniform-24      0.409897   35.095181          NaN     5.073461   
              uniform-26      1.517634  157.081932          NaN    22.515912   


### Speedup

### Create the output file

In [10]:
path = os.environ["HOME"] + "/workspace/sortledton/src/tables/analytics_speedup.tex"

In [11]:
def fmtsecs(value):
    if(value >= 20):
        return "{} s".format(int(value))
    else:
        return "{:.2f} s".format(value)

In [24]:
f = open(path, "w")
f.write(r"""
% This file was autogenerated by the script graphalytics_gen_table.ipynb

\resizebox{\textwidth}{!}{ % the table should use 100% of the available width
    \begin{tabular}[t]{clccccccc|clcccccccc}
    \toprule
        Graph  & System            & BFS       & CDLP       & LCC         & LCC (opt)   & PageRank     & SSSP        & WCC             
      & Graph  & System            & BFS       & CDLP       & LCC         & LCC (opt)   & PageRank     & SSSP        & WCC  \\   
    \midrule
""")

def write_csr(graph0, graph1, name0, name1, num_libraries):
    # first graph
    f.write("\\parbox[t]{2mm}{\\multirow{%s}{*}{\\rotatebox[origin=c]{90}{%s}}} & %% %s \n" % (num_libraries, name0, graph0)) 
    f.write("CSR (baseline) ")
    
    if True:
        d0 = ds.loc[("csr", graph0)]
        for c in d0: 
            if np.isnan(c):
                f.write("& DNF")
            else:   
                f.write("& $%s$ " % (fmtsecs(c)))
        f.write(" &\n")
    else:
        f.write(" & & & & & & & &")
        
    # second graph
    f.write("\\parbox[t]{2mm}{\\multirow{%s}{*}{\\rotatebox[origin=c]{90}{%s}}} & %% %s \n" % (num_libraries, name1, graph1)) 
    f.write("CSR (baseline) ") # without &
    d0 = ds.loc[("csr", graph1)]
    for c in d0: 
        if np.isnan(c):
            f.write("& DNF")
        else:       
            f.write("& $%s$ " % (fmtsecs(c)))
    f.write("\\\\\n")
    
def write_speedup_single0(library, graph):
    if True:
        completionTime = ds.loc[(library, graph)]
        speedup = completionTime  / ds.loc[("csr", graph)] 
        for column in speedup.index:
            f.write(" & ")
            value = speedup[column]
            best = ds.loc[("best_excl_csr", graph), column]
            if(np.isnan(value)):
                if column == "lcc_std" and (library == "sortledton" or library.startswith("teseo")):
                    f.write("N/A")
                elif column == "lcc_sorted" and not (library == "sortledton" or library.startswith("teseo")):
                    f.write("N/A")
                else:
                    f.write("DNF")
            elif completionTime[column] <= best:
                f.write("$\winner{");
                f.write("{:.2f}x".format(value))
                f.write("}$")
            else:
                f.write("${:.2f}x$".format(value))
    else:
        f.write(" & & & & & & & ")

def write_speedup_single(label, library, graph0, graph1):
    f.write(" & " + label)
    write_speedup_single0(library, graph0)
    f.write(" & & " + label)
    write_speedup_single0(library, graph1)
    f.write(" \\\\ \n")
    
def write_speedup(graph0, graph1, include_llama = True, include_stinger = True, include_graphone=True):
    if (include_stinger): write_speedup_single("Stinger", "stinger", graph0, graph1)
    if(include_llama): write_speedup_single("LLAMA", "llama", graph0, graph1)
    if (include_graphone): write_speedup_single("GraphOne", "graphone", graph0, graph1)
    write_speedup_single("LiveGraph", "livegraph", graph0, graph1)
    write_speedup_single("Teseo, log. vtx", "teseo_logical", graph0, graph1)
    if (graph0 != "com-friendster"): write_speedup_single("Teseo, real vtx", "teseo_real", graph0, graph1)
    write_speedup_single("Sortledton", "sortledton", graph0, graph1)
    
write_csr("dota-league", "graph500-22", "DOTA League", "Graph500 SF 22", 7)
write_speedup("dota-league", "graph500-22")
f.write("\\midrule \n")
write_csr("uniform-24", "graph500-24", "Uniform SF 24", "Graph500 SF 24", 7)
write_speedup("uniform-24", "graph500-24")
f.write("\\midrule \n")
write_csr("uniform-26", "graph500-26", "Uniform SF 26", "Graph500 SF 26", 6)
write_speedup("uniform-26", "graph500-26", include_llama = False, include_stinger = False)
f.write("\\midrule \n")
write_csr("com-friendster", "graph500-26", "Friendster", "Graph500 SF 26", 6)
write_speedup("com-friendster", "graph500-26", include_llama = False, include_stinger = False, include_graphone=False)
    
f.write(r"""
        \bottomrule
    \end{tabular}
} % end fo resizebox
""")
f.close()